# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [5]:
# Combine all features
X_train = pd.concat([X_train_numeric,X_train_cat],axis=1)
X_val = pd.concat([X_val_numeric,X_val_cat],axis=1)
X_test =pd.concat([X_test_numeric,X_test_cat],axis=1)

As a refresher, preview the training data: 

## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [7]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [12]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100,'relu',n_features))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50,'relu'))

# Output layer
baseline_model.add(layers.Dense(1,'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

                   
# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

33/33 [==============================] - 0s 12ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [============

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] -

Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [===========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [17]:
# Standardize the numeric features
ss_X = StandardScaler()

# Fit on training numeric data
X_train_scaled = ss_X.fit_transform(X_train_numeric)

# Transform validation and test numeric data
X_val_scaled = ss_X.transform(X_val_numeric)
X_test_scaled = ss_X.transform(X_test_numeric)

# Convert scaled arrays back to DataFrames and restore column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_numeric.columns, index=X_train_numeric.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val_numeric.columns, index=X_val_numeric.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_numeric.columns, index=X_test_numeric.index)

# Concatenate scaled numeric and categorical features (column-wise)
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [18]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [19]:
# Train the model
history = normalized_input_model.fit(
    X_train, 
    y_train, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val),
)

Epoch 1/150
33/33 [==============================] - 0s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [20]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_val)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [22]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
history = normalized_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled),
)

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 0.5503 - mse: 0.5503 - val_loss: 0.1860 - val_mse: 0.1860
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2383 - mse: 0.2383 - val_loss: 0.1514 - val_mse: 0.1514
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1916 - mse: 0.1916 - val_loss: 0.1296 - val_mse: 0.1296
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1622 - mse: 0.1622 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1413 - mse: 0.1413 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1365 - mse: 0.1365 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1231 - mse: 0.1231 - val_loss: 0.1231 - val_mse: 0.1231
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1129 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1488 - val_mse: 0.1488
Epoch 64/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0255 - mse: 0.0255 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1488 - val_mse: 0.1488
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1529 - val_mse: 0.1529
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1500 - val_mse: 0.1500
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1514 - val_mse: 0.1514
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1507 - val_mse: 0.1507
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0237 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0130 - mse: 0.0130 - val_loss: 0.1638 - val_mse: 0.1638
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1637 - val_mse: 0.1637
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1669 - val_mse: 0.1669
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1641 - val_mse: 0.1641
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1635 - val_mse: 0.1635
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1646 - val_mse: 0.1646
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1617 - val_mse: 0.1617
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
normalized_results = normalized_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099


- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
normalized_results_val = normalized_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1659 - mse: 0.1659


Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [28]:
# 1. Predict on validation set
y_val_pred_scaled = normalized_model.predict(X_val)

# 2. Inverse transform predictions
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# 3. Inverse transform y_val (convert to float first)
y_val_true = ss_y.inverse_transform(y_val.astype(float))

# 4. Compute RMSE
rmse = np.sqrt(mean_squared_error(y_val_true, y_val_pred))
print(f"RMSE on validation data: {rmse:.2f}")


RMSE on validation data: 14929429762.58


Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [29]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model(layers.Dense(100,activation='relu',input_shape=(n_features),kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
history = he_model.fit(
    X_train, 
    y_train_scaled, 
    batch_size=32, 
    epochs=150, 
    validation_data=(X_val, y_val_scaled),
)



Consider rewriting this model with the Functional API.
Epoch 1/150

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

33/33 [==============================] - 0s 7ms/step - loss: 0.4509 - mse: 0.4509 - val_loss: 0.2520 - val_mse: 0.2520
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2649 - mse: 0.2649 - val_loss: 0.1989 - val_mse: 0.1989
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2221 - mse: 0.2221 - val_loss: 0.1567 - val_mse: 0.1567
Epoch 4/150
33/33 [==============================] - 0s 3ms/

33/33 [==============================] - 0s 3ms/step - loss: 0.0476 - mse: 0.0476 - val_loss: 0.1566 - val_mse: 0.1566
Epoch 56/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0475 - mse: 0.0475 - val_loss: 0.1574 - val_mse: 0.1574
Epoch 57/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0458 - mse: 0.0458 - val_loss: 0.1630 - val_mse: 0.1630
Epoch 58/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0456 - mse: 0.0456 - val_loss: 0.1589 - val_mse: 0.1589
Epoch 59/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0449 - mse: 0.0449 - val_loss: 0.1712 - val_mse: 0.1712
Epoch 60/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0440 - mse: 0.0440 - val_loss: 0.1676 - val_mse: 0.1676
Epoch 61/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0448 - mse: 0.0448 - val_loss: 0.1616 - val_mse: 0.1616
Epoch 62/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0433 - mse: 0

33/33 [==============================] - 0s 3ms/step - loss: 0.0262 - mse: 0.0262 - val_loss: 0.1784 - val_mse: 0.1784
Epoch 118/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1776 - val_mse: 0.1776
Epoch 119/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1786 - val_mse: 0.1786
Epoch 120/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1803 - val_mse: 0.1803
Epoch 121/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1797 - val_mse: 0.1797
Epoch 122/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1783 - val_mse: 0.1783
Epoch 123/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1796 - val_mse: 0.1796
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0242 -

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
he_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0193 - mse: 0.0193


[0.019286666065454483, 0.019286666065454483]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [31]:
# Evaluate the model on validate data
he_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1841 - mse: 0.1841


[0.18414996564388275, 0.18414996564388275]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [32]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 0.4841 - mse: 0.4841 - val_loss: 0.1617 - val_mse: 0.1617
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2525 - mse: 0.2525 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2081 - mse: 0.2081 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1791 - mse: 0.1791 - val_loss: 0.1042 - val_mse: 0.1042
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1626 - mse: 0.1626 - val_loss: 0.0945 - val_mse: 0.0945
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1465 - mse: 0.1465 - val_loss: 0.0982 - val_mse: 0.0982
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1359 - mse: 0.1359 - val_loss: 0.0901 - val_mse: 0.0901
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1261 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1087 - val_mse: 0.1087
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1143 - val_mse: 0.1143
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1095 - val_mse: 0.1095
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1096 - val_mse: 0.1096
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1101 - val_mse: 0.1101
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1117 - val_mse: 0.1117
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1115 - val_mse: 0.1115
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1106 - val_mse: 0.1106
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1111 - val_mse: 0.1111
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
lecun_model.evaluate(X_train,y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0080 - mse: 0.0080


[0.008015928789973259, 0.008015928789973259]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val,y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1129 - mse: 0.1129


[0.11287495493888855, 0.11287495493888855]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [ ]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data


Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [ ]:
# Evaluate the model on validate data


## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [ ]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [ ]:
# Evaluate the model on training data


Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [ ]:
# Evaluate the model on validate data


## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [ ]:
# Evaluate the best model on test data


As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [ ]:
# Generate predictions on test data
y_test_pred_scaled = None

# Transform the predictions back to original scale
y_test_pred = None

# MSE of test data


## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.